# OpenStreetMap.org API 이용해서 위경도를 통한 도시코드 검색

참고자료:
1. http://daplus.net/geolocation-%EC%9C%84%EB%8F%84-%EA%B2%BD%EB%8F%84-%EC%A2%8C%ED%91%9C%EA%B0%80-%EC%A3%BC%EC%96%B4%EC%A7%80%EB%A9%B4-%EB%8F%84%EC%8B%9C-%EA%B5%AD%EA%B0%80%EB%A5%BC-%EC%96%B4%EB%96%BB%EA%B2%8C/

1. https://www.openstreetmap.org


1. https://geopy.readthedocs.io/en/stable/

오픈 소스 대안은 Open Street Map의 Nominatim입니다. URL에 변수를 설정하기 만하면 해당 위치의 도시 / 국가가 반환됩니다. 
공식 문서는 다음 링크를 확인하십시오 : [Nominatim](https://wiki.openstreetmap.org/wiki/Nominatim#Reverse_Geocoding)



### geopy 모듈를 이용해서 위경도 검색

[geppy](https://geopy.readthedocs.io/en/stable/)

[Nominatim Document](https://nominatim.org/release-docs/develop/api/Search/#parameters)

In [69]:
pip install geopy

Note: you may need to restart the kernel to use updated packages.


In [70]:
from geopy.geocoders import Nominatim

geolocator = Nominatim(user_agent="jeeedy")
location = geolocator.reverse("48.8588443, 2.2943506")
print("raw: ",location.raw)

raw:  {'place_id': 95534969, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'osm_type': 'way', 'osm_id': 5013364, 'lat': '48.858260200000004', 'lon': '2.2944990543196795', 'display_name': 'Tour Eiffel, 5, Avenue Anatole France, Quartier du Gros-Caillou, Paris 7e Arrondissement, Paris, Île-de-France, France métropolitaine, 75007, France', 'address': {'tourism': 'Tour Eiffel', 'house_number': '5', 'road': 'Avenue Anatole France', 'city_block': 'Quartier du Gros-Caillou', 'suburb': 'Paris 7e Arrondissement', 'city_district': 'Paris', 'county': 'Paris', 'state': 'Île-de-France', 'region': 'France métropolitaine', 'postcode': '75007', 'country': 'France', 'country_code': 'fr'}, 'boundingbox': ['48.8574753', '48.8590453', '2.2933084', '2.2956897']}


In [71]:
!pip install Elasticsearch

from elasticsearch import Elasticsearch

In [78]:
es = Elasticsearch("http://172.16.101.152:9200/")
print(es)

<Elasticsearch([{'host': '172.16.101.152', 'port': 9200}])>


- 총 호텔 갯수 = 354,500
- 10개 호텔 = 약 5 sec
- 10000 개 호텔 = 약 1.3 h (elasticsearch api 최대 검색 갯수)


In [82]:
import requests
import json
import datetime

def send_api(url, method, jsonBody={}):
    headers = {'Content-Type': 'application/json', 'charset': 'UTF-8', 'Accept': '*/*'}
    try:
        if method == 'GET':
            response = requests.get(url, headers=headers)
        elif method == 'POST':
            response = requests.post(url, headers=headers, data=json.dumps(jsonBody, ensure_ascii=False, indent="\t"))
        print("response status %r: %s" %(response.status_code, url)) 
        return response.text
    except Exception as ex:
        print(ex)
        
def reversGeocoder(lat, long, zoom):
    queryString = "https://nominatim.openstreetmap.org/reverse.php?lat=%s&lon=%s&zoom=%s&format=geojson&accept-language=en&extratags=1" %(lat, long, zoom)
    location = json.loads(send_api(queryString, "GET"))
    print(location,"\n")
    
    extratags = location["features"][0]["properties"]["extratags"]
    print("%s zooom extratags=%s" %(zoom, extratags))
    cityCode = extratags.get("ISO3166-2", None)
    print("%s zoom 도시코드: %s \n" %(zoom, cityCode))
    
    return cityCode


now = datetime.datetime.now()
print("# 시작 시간 %s" %now)

for cnt in range(1, 2):
    print("## cnt= %s" %cnt)
    query={
        "bool":{
            "filter":{
              "exists": {
                "field": "location"
              }
            },
            "must_not": {
              "exists": {
                "field": "iso3166-2"
              }
            }
        }
    }
    res = es.search(index="p-hotel-kimjy", size=10000, query=query)
    # print(res)

    for result in res["hits"]["hits"]:
        # print(result)
        id = result["_id"]
        sub_lat = result["_source"]["location"]["lat"]
        sub_long =  result["_source"]["location"]["lon"]
        title =  result["_source"]["name"]
        print("## geo Search start [id=%s, lat=%s, long=%s, title=%s] ##" %(id, sub_lat, sub_long, title) )

        if sub_lat==0 and sub_long==0 :
            continue

        # zoom 5 (state) 수준의 위치 검사
        cityCode = reversGeocoder(sub_lat, sub_long, 5)

        # zoom 6 (region) 수준의 위치 검사
        if cityCode is None:
            cityCode=reversGeocoder(sub_lat, sub_long, 6)

        # zoom 8 (county) 수준의 위치 검사
        if cityCode is None:
            cityCode=reversGeocoder(sub_lat, sub_long, 8)

        # zoom 10 (city) 수준의 위치 검사
        if cityCode is None:
            cityCode=reversGeocoder(sub_lat, sub_long, 10)

        if cityCode is not None:
            updatedoc={
                "doc":{
                    "iso3166-2": cityCode
                }
            }
            print("### Elasticsearch update ios3166-2 code  : %s ###" %cityCode)
            es.update(index="p-hotel-kimjy", id=id, body=updateBody)

        print("------ \n\n")
    
now = datetime.datetime.now()
print("# 종료 시간 %s" %now)


# 시작 시간 2022-01-21 01:25:51.933309
## cnt= 1
## geo Search start [id=7916, lat=41.99626, long=21.55224, title=호텔 벨뷰] ##
response status 200: https://nominatim.openstreetmap.org/reverse.php?lat=41.99626&lon=21.55224&zoom=5&format=geojson&accept-language=en&extratags=1
{'type': 'FeatureCollection', 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. https://osm.org/copyright', 'features': [{'type': 'Feature', 'properties': {'place_id': 282690705, 'osm_type': 'relation', 'osm_id': 2460455, 'place_rank': 8, 'category': 'boundary', 'type': 'administrative', 'importance': 0.40206279738389133, 'addresstype': 'state', 'name': 'Skopje Region', 'display_name': 'Skopje Region, North Macedonia', 'address': {'state': 'Skopje Region', 'country': 'North Macedonia', 'country_code': 'mk'}, 'extratags': {'wikidata': 'Q21115'}}, 'bbox': [21.1498886, 41.7037948, 21.8213797, 42.2491922], 'geometry': {'type': 'Point', 'coordinates': [21.462439778839435, 41.97656105]}}]} 

5 zooom extratags={'wikidata':